# Porter Logistics Case Study

**Context:**

Porter is India's Largest Marketplace for Intra-City Logistics. Leader in the country's $40 billion intra-city logistics market, Porter strives to improve the lives of 1,50,000+ driver-partners by providing them with consistent earning & independence. Currently, the company has serviced 5+ million customers

Porter works with a wide range of restaurants for delivering their items directly to the people.

Porter has a number of delivery partners available for delivering the food, from various restaurants and wants to get an estimated delivery time that it can provide the customers on the basis of what they are ordering, from where and also the delivery partners.

This dataset has the required data to train a regression model that will do the delivery time estimation, based on all those features


**Data Dictionary**

Each row in this file corresponds to one unique delivery. Each column corresponds to a feature as explained below.

1. market\_id : integer id for the market where the restaurant lies
2. created\_at : the timestamp at which the order was placed
3. actual\_delivery\_time : the timestamp when the order was delivered
4. store\_primary\_category : category for the restaurant
5. order\_protocol : integer code value for order protocol(how the order was placed ie: through porter, call to restaurant, pre booked, third part etc)
6. total\_items subtotal : final price of the order
7. num\_distinct\_items : the number of distinct items in the order
8. min\_item\_price : price of the cheapest item in the order
9. max\_item\_price : price of the costliest item in order
10. total\_onshift\_partners : number of delivery partners on duty at the time order was placed
11. total\_busy\_partners : number of delivery partners attending to other tasks
12. total\_outstanding\_orders : total number of orders to be fulfilled at the moment

In [1]:
import polars as pl
import duckdb as db
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [2]:
df = pl.read_csv("../data/raw/dataset.csv")

# Data Exploration

In [3]:
df.head()

market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
f64,str,str,str,str,f64,i64,i64,i64,i64,i64,f64,f64,f64
1.0,"""2015-02-06 22:24:17""","""2015-02-06 23:27:16""","""df263d996281d984952c07998dc543…","""american""",1.0,4,3441,4,557,1239,33.0,14.0,21.0
2.0,"""2015-02-10 21:49:25""","""2015-02-10 22:56:29""","""f0ade77b43923b38237db569b016ba…","""mexican""",2.0,1,1900,1,1400,1400,1.0,2.0,2.0
3.0,"""2015-01-22 20:39:28""","""2015-01-22 21:09:09""","""f0ade77b43923b38237db569b016ba…",null,1.0,1,1900,1,1900,1900,1.0,0.0,0.0
3.0,"""2015-02-03 21:21:45""","""2015-02-03 22:13:00""","""f0ade77b43923b38237db569b016ba…",null,1.0,6,6900,5,600,1800,1.0,1.0,2.0
3.0,"""2015-02-15 02:40:36""","""2015-02-15 03:20:26""","""f0ade77b43923b38237db569b016ba…",null,1.0,3,3900,3,1100,1600,6.0,6.0,9.0


In [4]:
df.shape

(197428, 14)

In [5]:
df.describe()

statistic,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
str,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",196441.0,"""197428""","""197421""","""197428""","""192668""",196433.0,197428.0,197428.0,197428.0,197428.0,197428.0,181166.0,181166.0,181166.0
"""null_count""",987.0,"""0""","""7""","""0""","""4760""",995.0,0.0,0.0,0.0,0.0,0.0,16262.0,16262.0,16262.0
"""mean""",2.978706,null,null,null,null,2.882352,3.196391,2682.331402,2.670791,686.21847,1159.58863,44.808093,41.739747,58.050065
"""std""",1.524867,null,null,null,null,1.503771,2.666546,1823.093688,1.630255,522.038648,558.411377,34.526783,32.145733,52.66183
"""min""",1.0,"""2014-10-19 05:24:15""","""2015-01-21 15:58:11""","""0004d0b59e19461ff126e3a08a814c…","""afghan""",1.0,1.0,0.0,1.0,-86.0,0.0,-4.0,-5.0,-6.0
"""25%""",2.0,null,null,null,null,1.0,2.0,1400.0,1.0,299.0,800.0,17.0,15.0,17.0
"""50%""",3.0,null,null,null,null,3.0,3.0,2200.0,2.0,595.0,1095.0,37.0,34.0,41.0
"""75%""",4.0,null,null,null,null,4.0,4.0,3395.0,3.0,949.0,1395.0,65.0,62.0,85.0
"""max""",6.0,"""2015-02-18 06:00:44""","""2015-02-19 22:45:31""","""ffedf5be3a86e2ee281d54cdc97bc1…","""vietnamese""",7.0,411.0,27100.0,20.0,14700.0,14700.0,171.0,154.0,285.0


In [6]:
dict(zip(df.columns, df.dtypes))

{'market_id': Float64,
 'created_at': String,
 'actual_delivery_time': String,
 'store_id': String,
 'store_primary_category': String,
 'order_protocol': Float64,
 'total_items': Int64,
 'subtotal': Int64,
 'num_distinct_items': Int64,
 'min_item_price': Int64,
 'max_item_price': Int64,
 'total_onshift_partners': Float64,
 'total_busy_partners': Float64,
 'total_outstanding_orders': Float64}

In [7]:
df.null_count()

market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
987,0,7,0,4760,995,0,0,0,0,0,16262,16262,16262


In [8]:
db.sql(""" 
select * from df where total_onshift_partners IS NULL limit 20
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │ min_item_price │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │     int64      │     int64      │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼────────────

In [9]:
db.sql(""" 
select * from df where store_primary_category IS NULL limit 20
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │ min_item_price │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │     int64      │     int64      │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼────────────

In [10]:
db.sql(""" 
select * from df where order_protocol IS NULL limit 20
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │ min_item_price │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │     int64      │     int64      │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼────────────

In [11]:
db.sql(""" 
select market_id, store_primary_category, count(*) from df group by market_id, store_primary_category order by count(*) desc limit 20
""")

┌───────────┬────────────────────────┬──────────────┐
│ market_id │ store_primary_category │ count_star() │
│  double   │        varchar         │    int64     │
├───────────┼────────────────────────┼──────────────┤
│       2.0 │ mexican                │         6647 │
│       2.0 │ american               │         5700 │
│       1.0 │ american               │         5228 │
│       2.0 │ pizza                  │         4859 │
│       1.0 │ pizza                  │         4081 │
│       4.0 │ pizza                  │         3840 │
│       2.0 │ sandwich               │         3751 │
│       2.0 │ burger                 │         3340 │
│       4.0 │ mexican                │         3250 │
│       2.0 │ dessert                │         3115 │
│       4.0 │ chinese                │         2847 │
│       3.0 │ american               │         2743 │
│       1.0 │ japanese               │         2628 │
│       4.0 │ burger                 │         2603 │
│       4.0 │ dessert       

In [12]:
df["store_primary_category"].value_counts().sort("count", descending=True)

store_primary_category,count
str,u32
"""american""",19399
"""pizza""",17321
"""mexican""",17099
"""burger""",10958
"""sandwich""",10060
…,…
"""lebanese""",9
"""belgian""",2
"""indonesian""",2


In [13]:
df["order_protocol"].value_counts().sort("count", descending=True)

order_protocol,count
f64,u32
1.0,54725
3.0,53199
5.0,44290
2.0,24052
4.0,19354
null,995
6.0,794
7.0,19


In [14]:
df["market_id"].value_counts().sort("count", descending=True)

market_id,count
f64,u32
2.0,55058
4.0,47599
1.0,38037
3.0,23297
5.0,18000
6.0,14450
null,987


In [15]:
df=df.with_columns(
    pl.col("min_item_price").abs(),
    pl.col("max_item_price").abs(),
    pl.col("total_onshift_partners").abs(),
    pl.col("total_busy_partners").abs(),
    pl.col("total_outstanding_orders").abs(),
)

In [16]:
df.describe()

statistic,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
str,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",196441.0,"""197428""","""197421""","""197428""","""192668""",196433.0,197428.0,197428.0,197428.0,197428.0,197428.0,181166.0,181166.0,181166.0
"""null_count""",987.0,"""0""","""7""","""0""","""4760""",995.0,0.0,0.0,0.0,0.0,0.0,16262.0,16262.0,16262.0
"""mean""",2.978706,null,null,null,null,2.882352,3.196391,2682.331402,2.670791,686.222339,1159.58863,44.808524,41.740166,58.051433
"""std""",1.524867,null,null,null,null,1.503771,2.666546,1823.093688,1.630255,522.033561,558.411377,34.526225,32.145188,52.660321
"""min""",1.0,"""2014-10-19 05:24:15""","""2015-01-21 15:58:11""","""0004d0b59e19461ff126e3a08a814c…","""afghan""",1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"""25%""",2.0,null,null,null,null,1.0,2.0,1400.0,1.0,299.0,800.0,17.0,15.0,17.0
"""50%""",3.0,null,null,null,null,3.0,3.0,2200.0,2.0,595.0,1095.0,37.0,34.0,41.0
"""75%""",4.0,null,null,null,null,4.0,4.0,3395.0,3.0,949.0,1395.0,65.0,62.0,85.0
"""max""",6.0,"""2015-02-18 06:00:44""","""2015-02-19 22:45:31""","""ffedf5be3a86e2ee281d54cdc97bc1…","""vietnamese""",7.0,411.0,27100.0,20.0,14700.0,14700.0,171.0,154.0,285.0


In [17]:
df.null_count()

market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
987,0,7,0,4760,995,0,0,0,0,0,16262,16262,16262


# Null Values Imputation

In [18]:
df = df.drop_nulls(subset=["total_onshift_partners", "total_busy_partners", "total_outstanding_orders", "actual_delivery_time",\
                        #    "market_id","store_primary_category","order_protocol"\
                           ])

In [19]:
df.describe()

statistic,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
str,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",180240.0,"""181159""","""181159""","""181159""","""176944""",180242.0,181159.0,181159.0,181159.0,181159.0,181159.0,181159.0,181159.0,181159.0
"""null_count""",919.0,"""0""","""0""","""0""","""4215""",917.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2.748702,null,null,null,null,2.895923,3.208469,2698.524296,2.677659,684.854095,1160.752698,44.807296,41.739207,58.049338
"""std""",1.331715,null,null,null,null,1.514983,2.673052,1829.304441,1.627291,521.286542,561.838633,34.525354,32.144618,52.658547
"""min""",1.0,"""2015-01-21 15:22:03""","""2015-01-21 15:58:11""","""0004d0b59e19461ff126e3a08a814c…","""afghan""",1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"""25%""",2.0,null,null,null,null,1.0,2.0,1415.0,2.0,299.0,799.0,17.0,15.0,17.0
"""50%""",2.0,null,null,null,null,3.0,3.0,2225.0,2.0,595.0,1095.0,37.0,34.0,41.0
"""75%""",4.0,null,null,null,null,4.0,4.0,3411.0,3.0,942.0,1395.0,65.0,62.0,85.0
"""max""",6.0,"""2015-02-18 06:00:44""","""2015-02-19 22:45:31""","""ffeabd223de0d4eacb9a3e6e53e544…","""vietnamese""",7.0,411.0,26800.0,20.0,14700.0,14700.0,171.0,154.0,285.0


In [20]:
db.sql("""
    select * from df where max_item_price = 0 and min_item_price = 0
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │ min_item_price │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │     int64      │     int64      │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼────────────

In [21]:
df=df.with_columns(
    pl.when(pl.col("max_item_price") == 0).then(pl.col("max_item_price").mean()).otherwise(pl.col("max_item_price")).alias("max_item_price"),
    pl.when(pl.col("min_item_price") == 0).then(pl.col("min_item_price").mean()).otherwise(pl.col("min_item_price")).alias("min_item_price")
)

In [22]:
db.sql("""
    select * from df where max_item_price < min_item_price
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬───────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │  min_item_price   │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │      double       │     double     │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼───

In [23]:
df=df.with_columns(
    max_price_temp=pl.col('max_item_price'),
    min_price_temp=pl.col('min_item_price'),
)

In [24]:
df=df.with_columns(
    max_item_price=pl.when(pl.col('max_price_temp') < pl.col('min_price_temp'))
    .then(pl.col('min_price_temp'))
    .otherwise(pl.col('max_price_temp')),
    min_item_price=pl.when(pl.col('min_price_temp') > pl.col('max_price_temp'))
    .then(pl.col('max_price_temp'))
    .otherwise(pl.col('min_price_temp'))
).drop('max_price_temp', 'min_price_temp')

In [25]:
db.sql("""
    select * from df where  total_onshift_partners = 0 
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │ min_item_price │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │     double     │     double     │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼────────────

In [26]:
db.sql("""
    select * from df where total_busy_partners = 0 and total_onshift_partners = 0 and total_outstanding_orders = 0
""")

┌───────────┬─────────────────────┬──────────────────────┬──────────────────────────────────┬────────────────────────┬────────────────┬─────────────┬──────────┬────────────────────┬────────────────┬────────────────┬────────────────────────┬─────────────────────┬──────────────────────────┐
│ market_id │     created_at      │ actual_delivery_time │             store_id             │ store_primary_category │ order_protocol │ total_items │ subtotal │ num_distinct_items │ min_item_price │ max_item_price │ total_onshift_partners │ total_busy_partners │ total_outstanding_orders │
│  double   │       varchar       │       varchar        │             varchar              │        varchar         │     double     │    int64    │  int64   │       int64        │     double     │     double     │         double         │       double        │          double          │
├───────────┼─────────────────────┼──────────────────────┼──────────────────────────────────┼────────────────────────┼────────────

In [27]:
df=df.filter(
    ~(pl.col("total_onshift_partners") == 0) 
)

In [28]:
df.describe()

statistic,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
str,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",176642.0,"""177544""","""177544""","""177544""","""173383""",176640.0,177544.0,177544.0,177544.0,177544.0,177544.0,177544.0,177544.0,177544.0
"""null_count""",902.0,"""0""","""0""","""0""","""4161""",904.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2.767864,null,null,null,null,2.898687,3.205588,2701.713919,2.675618,692.093454,1164.986471,45.719624,42.58813,59.228873
"""std""",1.329898,null,null,null,null,1.513229,2.67464,1829.211488,1.62441,516.496735,559.219301,34.271843,31.909024,52.532368
"""min""",1.0,"""2015-01-21 15:22:03""","""2015-01-21 15:58:11""","""0004d0b59e19461ff126e3a08a814c…","""afghan""",1.0,1.0,0.0,1.0,1.0,52.0,1.0,0.0,0.0
"""25%""",2.0,null,null,null,null,1.0,2.0,1420.0,2.0,300.0,799.0,18.0,16.0,18.0
"""50%""",2.0,null,null,null,null,3.0,3.0,2228.0,2.0,599.0,1095.0,38.0,35.0,42.0
"""75%""",4.0,null,null,null,null,4.0,4.0,3421.0,3.0,943.0,1397.0,66.0,63.0,86.0
"""max""",6.0,"""2015-02-18 06:00:44""","""2015-02-19 22:45:31""","""ffeabd223de0d4eacb9a3e6e53e544…","""vietnamese""",7.0,411.0,26800.0,20.0,14700.0,14700.0,171.0,154.0,285.0


In [29]:
category_per_store = df.group_by("store_id").agg(
    pl.col("store_primary_category").mode().first().alias("store_primary_category_name")
)
category_per_store = category_per_store.fill_null("other")

In [30]:
df = df.join(category_per_store, on="store_id", how="left")

In [31]:
df = df.with_columns(
   pl.when(pl.col("store_primary_category").is_null()).then(pl.col("store_primary_category_name")).otherwise(pl.col("store_primary_category")).alias("store_primary_category")
).drop("store_primary_category_name")

In [32]:
market_id_per_store = df.group_by("store_id").agg(
    pl.col("market_id").mode().first().alias("market_id_num")
)
market_id_per_store = market_id_per_store.fill_null(pl.col("market_id_num").mode().first())

In [33]:
df = df.join(market_id_per_store, on="store_id", how="left")

In [34]:
df = df.with_columns(
   pl.when(pl.col("market_id").is_null()).then(pl.col("market_id_num")).otherwise(pl.col("market_id")).alias("market_id")
).drop("market_id_num")

In [35]:
df.null_count()

market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,904,0,0,0,0,0,0,0,0


In [36]:
df = df.with_columns(
    pl.col("created_at").str.to_datetime(),
    pl.col("actual_delivery_time").str.to_datetime()
)

**Time based feature generation**

In [40]:
df=df.with_columns(
    pl.col("created_at").dt.offset_by('-5h30m'),
    pl.col("actual_delivery_time").dt.offset_by('-5h30m')
)

In [41]:
df = df.with_columns(
    pl.col("created_at").dt.year().alias("created_at_year"),
    pl.col("created_at").dt.month().alias("created_at_month"),
    pl.col("created_at").dt.day().alias("created_at_day"),
    pl.col("created_at").dt.hour().alias("created_at_hour"),
    pl.col("created_at").dt.minute().alias("created_at_minute"),
    pl.col("created_at").dt.second().alias("created_at_second"),
    pl.col("actual_delivery_time").dt.year().alias("actual_delivery_time_year"),
    pl.col("actual_delivery_time").dt.month().alias("actual_delivery_time_month"),
    pl.col("actual_delivery_time").dt.day().alias("actual_delivery_time_day"),
    pl.col("actual_delivery_time").dt.hour().alias("actual_delivery_time_hour"),
    pl.col("actual_delivery_time").dt.minute().alias("actual_delivery_time_minute"),
    pl.col("actual_delivery_time").dt.second().alias("actual_delivery_time_second")
)

In [42]:
df

market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders,created_at_year,created_at_month,created_at_day,created_at_hour,created_at_minute,created_at_second,actual_delivery_time_year,actual_delivery_time_month,actual_delivery_time_day,actual_delivery_time_hour,actual_delivery_time_minute,actual_delivery_time_second
f64,datetime[μs],datetime[μs],str,str,f64,i64,i64,i64,f64,f64,f64,f64,f64,i32,i8,i8,i8,i8,i8,i32,i8,i8,i8,i8,i8
1.0,2015-02-06 16:54:17,2015-02-06 17:57:16,"""df263d996281d984952c07998dc543…","""american""",1.0,4,3441,4,557.0,1239.0,33.0,14.0,21.0,2015,2,6,16,54,17,2015,2,6,17,57,16
2.0,2015-02-10 16:19:25,2015-02-10 17:26:29,"""f0ade77b43923b38237db569b016ba…","""mexican""",2.0,1,1900,1,1400.0,1400.0,1.0,2.0,2.0,2015,2,10,16,19,25,2015,2,10,17,26,29
3.0,2015-01-22 15:09:28,2015-01-22 15:39:09,"""f0ade77b43923b38237db569b016ba…","""other""",1.0,1,1900,1,1900.0,1900.0,1.0,0.0,0.0,2015,1,22,15,9,28,2015,1,22,15,39,9
3.0,2015-02-03 15:51:45,2015-02-03 16:43:00,"""f0ade77b43923b38237db569b016ba…","""other""",1.0,6,6900,5,600.0,1800.0,1.0,1.0,2.0,2015,2,3,15,51,45,2015,2,3,16,43,0
3.0,2015-02-14 21:10:36,2015-02-14 21:50:26,"""f0ade77b43923b38237db569b016ba…","""other""",1.0,3,3900,3,1100.0,1600.0,6.0,6.0,9.0,2015,2,14,21,10,36,2015,2,14,21,50,26
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1.0,2015-02-16 18:49:41,2015-02-16 19:54:48,"""a914ecef9c12ffdb9bede64bb703d8…","""fast""",4.0,3,1389,3,345.0,649.0,17.0,17.0,23.0,2015,2,16,18,49,41,2015,2,16,19,54,48
1.0,2015-02-12 18:31:59,2015-02-12 19:28:22,"""a914ecef9c12ffdb9bede64bb703d8…","""fast""",4.0,6,3010,4,405.0,825.0,12.0,11.0,14.0,2015,2,12,18,31,59,2015,2,12,19,28,22
1.0,2015-01-23 23:16:08,2015-01-24 00:06:16,"""a914ecef9c12ffdb9bede64bb703d8…","""fast""",4.0,5,1836,3,300.0,399.0,39.0,41.0,40.0,2015,1,23,23,16,8,2015,1,24,0,6,16


In [43]:
cols=[
 'order_protocol',
 'total_items',
 'subtotal',
 'num_distinct_items',
 'min_item_price',
 'max_item_price',
 'total_onshift_partners',
 'total_busy_partners',
 'total_outstanding_orders',
 'created_at_year',
 'created_at_month',
 'created_at_day',
 'created_at_hour',
 'created_at_minute',
 'actual_delivery_time_year',
 'actual_delivery_time_month',
 'actual_delivery_time_day',
 'actual_delivery_time_hour',
 'actual_delivery_time_minute'
 ]

In [44]:
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(df[cols].to_numpy())
knn_imputer = KNNImputer(n_neighbors=3)
imputed_features = knn_imputer.fit_transform(df[cols].to_numpy())

In [45]:
# imputed_features = scaler.inverse_transform(imputed_features)
imputed_features = pl.DataFrame(imputed_features)
imputed_features.columns = cols
imputed_features

order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders,created_at_year,created_at_month,created_at_day,created_at_hour,created_at_minute,actual_delivery_time_year,actual_delivery_time_month,actual_delivery_time_day,actual_delivery_time_hour,actual_delivery_time_minute
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.0,4.0,3441.0,4.0,557.0,1239.0,33.0,14.0,21.0,2015.0,2.0,6.0,16.0,54.0,2015.0,2.0,6.0,17.0,57.0
2.0,1.0,1900.0,1.0,1400.0,1400.0,1.0,2.0,2.0,2015.0,2.0,10.0,16.0,19.0,2015.0,2.0,10.0,17.0,26.0
1.0,1.0,1900.0,1.0,1900.0,1900.0,1.0,0.0,0.0,2015.0,1.0,22.0,15.0,9.0,2015.0,1.0,22.0,15.0,39.0
1.0,6.0,6900.0,5.0,600.0,1800.0,1.0,1.0,2.0,2015.0,2.0,3.0,15.0,51.0,2015.0,2.0,3.0,16.0,43.0
1.0,3.0,3900.0,3.0,1100.0,1600.0,6.0,6.0,9.0,2015.0,2.0,14.0,21.0,10.0,2015.0,2.0,14.0,21.0,50.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4.0,3.0,1389.0,3.0,345.0,649.0,17.0,17.0,23.0,2015.0,2.0,16.0,18.0,49.0,2015.0,2.0,16.0,19.0,54.0
4.0,6.0,3010.0,4.0,405.0,825.0,12.0,11.0,14.0,2015.0,2.0,12.0,18.0,31.0,2015.0,2.0,12.0,19.0,28.0
4.0,5.0,1836.0,3.0,300.0,399.0,39.0,41.0,40.0,2015.0,1.0,23.0,23.0,16.0,2015.0,1.0,24.0,0.0,6.0


In [46]:
df=df.with_columns(
    pl.when(pl.col("order_protocol").is_null())
    .then(imputed_features["order_protocol"])
    .otherwise(pl.col("order_protocol"))
    .cast(pl.Int8)
    .alias("order_protocol")
)

In [47]:
df.write_parquet("../data/cleaned/dataset.parquet")

# Feature Generation

In [54]:
df=pl.read_parquet("../data/cleaned/dataset.parquet")

In [55]:
df["created_at"].max(), df["created_at"].min()

(datetime.datetime(2015, 2, 18, 0, 30, 44),
 datetime.datetime(2015, 1, 21, 9, 52, 3))

In [56]:
df=df.with_columns(
    pl.col("created_at").dt.strftime("%b").alias("created_at_month_name"),
    pl.col("actual_delivery_time").dt.strftime("%b").alias("actual_delivery_time_month_name"),
    pl.col("created_at").dt.weekday().alias("created_at_day_of_week"),
    pl.col("actual_delivery_time").dt.weekday().alias("actual_delivery_time_day_of_week"),
    pl.col("created_at").dt.strftime("%a").alias("created_at_day_name"),
    pl.col("actual_delivery_time").dt.strftime("%a").alias("actual_delivery_time_day_name"),
    pl.col("created_at").dt.week().alias("created_at_week_number"),
    pl.col("actual_delivery_time").dt.week().alias("actual_delivery_time_week_number"),
   ((pl.col("created_at").dt.day() - 1) // 7 + 1).cast(pl.Int16).alias("created_at_week_of_month"),
    ((pl.col("actual_delivery_time").dt.day() - 1) // 7 + 1).cast(pl.Int16).alias("actual_delivery_time_week_of_month")

)

In [57]:
df=df.with_columns(
    ((pl.col("actual_delivery_time") - pl.col("created_at")).dt.total_seconds())
    .alias("delivery_time_seconds"),
    ((pl.col("actual_delivery_time") - pl.col("created_at")).dt.total_minutes())
    .alias("delivery_time_minutes"),
)

In [58]:
df.with_columns(
    pl.col("market_id").cast(pl.Int8).alias("market_id"),
    pl.col("total_items").cast(pl.Int16).alias("total_items"),
    pl.col("subtotal").cast(pl.Int16).alias("subtotal"),
    pl.col("num_distinct_items").cast(pl.Int16).alias("num_distinct_items"),
    pl.col("min_item_price").cast(pl.Int16).alias("min_item_price"),
    pl.col("max_item_price").cast(pl.Int16).alias("max_item_price"),
)

market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders,created_at_year,created_at_month,created_at_day,created_at_hour,created_at_minute,created_at_second,actual_delivery_time_year,actual_delivery_time_month,actual_delivery_time_day,actual_delivery_time_hour,actual_delivery_time_minute,actual_delivery_time_second,created_at_month_name,actual_delivery_time_month_name,created_at_day_of_week,actual_delivery_time_day_of_week,created_at_day_name,actual_delivery_time_day_name,created_at_week_number,actual_delivery_time_week_number,created_at_week_of_month,actual_delivery_time_week_of_month,delivery_time_seconds,delivery_time_minutes
i8,datetime[μs],datetime[μs],str,str,i8,i16,i16,i16,i16,i16,f64,f64,f64,i32,i8,i8,i8,i8,i8,i32,i8,i8,i8,i8,i8,str,str,i8,i8,str,str,i8,i8,i16,i16,i64,i64
1,2015-02-06 16:54:17,2015-02-06 17:57:16,"""df263d996281d984952c07998dc543…","""american""",1,4,3441,4,557,1239,33.0,14.0,21.0,2015,2,6,16,54,17,2015,2,6,17,57,16,"""Feb""","""Feb""",5,5,"""Fri""","""Fri""",6,6,1,1,3779,62
2,2015-02-10 16:19:25,2015-02-10 17:26:29,"""f0ade77b43923b38237db569b016ba…","""mexican""",2,1,1900,1,1400,1400,1.0,2.0,2.0,2015,2,10,16,19,25,2015,2,10,17,26,29,"""Feb""","""Feb""",2,2,"""Tue""","""Tue""",7,7,2,2,4024,67
3,2015-01-22 15:09:28,2015-01-22 15:39:09,"""f0ade77b43923b38237db569b016ba…","""other""",1,1,1900,1,1900,1900,1.0,0.0,0.0,2015,1,22,15,9,28,2015,1,22,15,39,9,"""Jan""","""Jan""",4,4,"""Thu""","""Thu""",4,4,4,4,1781,29
3,2015-02-03 15:51:45,2015-02-03 16:43:00,"""f0ade77b43923b38237db569b016ba…","""other""",1,6,6900,5,600,1800,1.0,1.0,2.0,2015,2,3,15,51,45,2015,2,3,16,43,0,"""Feb""","""Feb""",2,2,"""Tue""","""Tue""",6,6,1,1,3075,51
3,2015-02-14 21:10:36,2015-02-14 21:50:26,"""f0ade77b43923b38237db569b016ba…","""other""",1,3,3900,3,1100,1600,6.0,6.0,9.0,2015,2,14,21,10,36,2015,2,14,21,50,26,"""Feb""","""Feb""",6,6,"""Sat""","""Sat""",7,7,2,2,2390,39
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,2015-02-16 18:49:41,2015-02-16 19:54:48,"""a914ecef9c12ffdb9bede64bb703d8…","""fast""",4,3,1389,3,345,649,17.0,17.0,23.0,2015,2,16,18,49,41,2015,2,16,19,54,48,"""Feb""","""Feb""",1,1,"""Mon""","""Mon""",8,8,3,3,3907,65
1,2015-02-12 18:31:59,2015-02-12 19:28:22,"""a914ecef9c12ffdb9bede64bb703d8…","""fast""",4,6,3010,4,405,825,12.0,11.0,14.0,2015,2,12,18,31,59,2015,2,12,19,28,22,"""Feb""","""Feb""",4,4,"""Thu""","""Thu""",7,7,2,2,3383,56
1,2015-01-23 23:16:08,2015-01-24 00:06:16,"""a914ecef9c12ffdb9bede64bb703d8…","""fast""",4,5,1836,3,300,399,39.0,41.0,40.0,2015,1,23,23,16,8,2015,1,24,0,6,16,"""Jan""","""Jan""",5,6,"""Fri""","""Sat""",4,4,4,4,3008,50


In [59]:
df.write_parquet("../data/cleaned/dataset.parquet")